# get pageviews with AWS

#### how to do jupyter with aws

- https://dataschool.com/data-modeling-101/running-jupyter-notebook-on-an-ec2-server/
    - except jupyter_notebook_config.py_ should be ...config.py instead
- https://gist.github.com/J535D165/0e840291e7b2598ec157e13e9b9ca569
    - trying this for how to use nohup
- some medium [article](https://medium.com/@christinakouride/a-beginners-guide-to-running-jupyter-notebook-on-amazon-ec2-69e1b74e73cc#:~:text=Your%20Jupyter%20Notebook%20server%20will,for%20time%20not%20using%20it.)
    - they pointed out that notebook will keep running, but didn't mention nohup

In [50]:
import os, requests, gzip, pickle
import pandas as pd
from datetime import datetime as dt

In [36]:
from platform import python_version
print(python_version())

3.9.7


## get page_titles and urls

###### unpickle page title list

In [ ]:
with open(processed_path + 'disaster_descendants_raw.pickle', 'rb') as f:
    disaster_descendants_raw = pickle.load(f)

## functions

##### function download_file

In [26]:
def download_file(url, dirpath='./'):
    local_filepath = dirpath + url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=131072):   #8192
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filepath
# https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests

###### function ```open_hour_file``` to provide the unzipped file to python

In [90]:
def open_hour_file(path:str):
    if path[-3:] == ".gz":
        return gzip.open(path, mode="rt", encoding="utf-8", errors="replace")
    else:
        return open(path, mode="rt", encoding="utf-8", errors="replace")
# https://github.com/mediawiki-utilities/python-mwviews/blob/main/src/mwviews/utilities/aggregate.py
# https://stackoverflow.com/questions/30582162/creating-a-missing-directory-file-structure-python

###### function ```subset_pageviews_by_page_titles``` to clean and filter pageview records

In [94]:
def subset_pageviews_by_page_titles(fpath_in, fpath_out, page_titles, proj='ja'):
    os.makedirs(os.path.dirname(fpath_out), exist_ok=True)
    with (open_hour_file(fpath_in) as f_in, 
          open(fpath_out, 'w', encoding='utf-8') as f_out
         ):
        while (line := f_in.readline()):
            if line[:3] != proj + ' ':
                continue
            lst = line.strip().split(sep=' ')
            if lst[1] in page_titles:
                f_out.write(line)

## get the files

###### get many urls

In [ ]:
'https://dumps.wikimedia.org/other/pageviews/2021/2021-01/pageviews-20210101-000000.gz'

In [96]:
ys = list(range(2021,2022))
hour_strings = list(map(lambda x: str(x).zfill(2), range(0,24)))
dates = pd.date_range(dt(ys[0],1,1), end=dt(ys[-1]+1,1,1), freq='D')  # , tz='Japan'
hs = map(lambda x: str(x).zfill(2), range(0,24))
base_url = 'https://dumps.wikimedia.org/other/pageviews/'
out_dirpath = '../data/processed/pageviews/'

urls = [f'{base_url}{d.year}/{d.year}-{str(d.month).zfill(2)}'
        f'/pageviews-{d.year}{str(d.month).zfill(2)}{str(d.day).zfill(2)}-{h}0000.gz'
        for d in dates for h in hour_strings]
outpaths = [f'{out_dirpath}{d.year}/{d.year}-{str(d.month).zfill(2)}'
        f'/pageviews-{d.year}{str(d.month).zfill(2)}{str(d.day).zfill(2)}-{h}0000.txt'
        for d in dates for h in hour_strings]

##### download many files

In [98]:
os.listdir('../data/processed/pageviews/2021/2021-01/')

[]

In [ ]:
start_time = datetime.datetime.now()
for url, outpath in zip(urls, outpaths):
    temp_fpath = download_file(url, dirpath=temp_dirpath)
    subset_pageviews_by_page_titles(temp_fpath,outpath,['台風'])
end_time = datetime.datetime.now()
print(end_time - start_time)

###### peek

In [ ]:
# !jupyter nbconvert --to script '1.22-sfb-get-pageviews-with-AWS-lambda.ipynb'

##### -----

###### ------

##### test running cell while laptop sleeping

###### idea

- consider the case where:
    - i start the following cell's script as follows:
        - on AWS from a ssh terminal 
        - via nohup jupyter on computer A
    - then i disconnect the ssh from computer A
        - so that the cell is still running in the instance
- then i find the result that:
    - i can reopen jupyter from another ssh terminal
    - i can interrupt the script
    - but i can't see its in-process results
- strategy for now:
    - develop the script on whatever computers
    - then run it from a browser tab in my android phone
        - that way i can keep "tabs" on it (😉)

###### test code

In [ ]:
from IPython.display import clear_output
import time

for i in range(600):
    print(i)
    time.sleep(1)
    clear_output(wait=True)